In [ ]:
import numpy as np
import os
import torch.nn as nn
import torch

In [ ]:
def readable(byte_int: int):
  """
  representing an integer in the byte, KB, MB etc. notation.
  """
  for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
    if byte_int < 1024:
      return f"{byte_int:.2f} {unit}"
    byte_int /= 1024

In [ ]:
def get_batch(dir: str):
  for file_name in os.listdir(dir):
    full_path = os.path.join(dir, file_name)
    if os.path.isfile(full_path):
      yield full_path

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self,
                 d_model: int,
                 eps: float = 1e-5,
                 device: str ='cuda'):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(d_model, device=device))


    def forward(self, x):
        output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) * self.weight

        return output

In [ ]:
def accuracy(predicted_labels, true_labels):
    correct = (predicted_labels == true_labels).sum().item()
    return correct / len(true_labels)

In [ ]:
def pad(X, y):
    """
    Padding the entire batch.
    X: sample (shape: [batch_size, seq_len_X, feature_dim])
    y: target (shape: [batch_size, seq_len_y, feature_dim])
    """
    batch_size, seq_len_X, feature_dim_X = X.shape
    _, seq_len_y, feature_dim_y = y.shape

    if seq_len_X != seq_len_y:
        # Find the difference in sequence lengths
        m = np.abs(seq_len_X - seq_len_y)

        # If X is shorter, pad X
        if seq_len_X < seq_len_y:
            pad_shape = (batch_size, m, feature_dim_X)
            p = np.zeros(pad_shape)
            X = np.concatenate((X, p), axis=1)
            X = X.astype(np.float32)
        # If y is shorter, pad y
        else:
            pad_shape = (batch_size, m, feature_dim_y)
            p = np.zeros(pad_shape)
            y = np.concatenate((y, p), axis=1)
            y = y.astype(np.float32)

    return X, y
